In [1]:
from wellplate.elements import read_plate_xml

well_map, info, labels = read_plate_xml('C:\Johan-test.xml')
display(well_map)

,well,EP,AB,Note
0,A1,A,AB2,Testing
1,A2,A,AB2,Testing
2,A3,A,AB2,Testing
3,A4,A,AB2,Testing
4,A5,A,AB2,Testing
...,...,...,...,...
91,H8,B,AB1,None
92,H9,B,AB2,None
93,H10,B,AB2,None
94,H11,B,AB2,None
